# REMEMBER: THE GOAL IS PASS THE THESIS IN 10 DAYS, NOT TRYING TO PREDICT THE BEST
- CORRECT DATA
- CORRECT METHOD
- MINIMUM EFFORT AND TIME
- A LOT OF EXPLANATION

# IMPORT

In [89]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.experimental import enable_hist_gradient_boosting  # Explicitly require this experimental feature
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import KFold, cross_validate
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import TimeSeriesSplit
from sklearn.compose import ColumnTransformer

# 1. LOAD DATA
- TRAIN: UNION OF ALL DATAFRAMES (UNIQUE = KEY + TIMESTAMP)
- TEST: NOT UNION BUT SEPERATE: DF10 ONLY

In [5]:
with open('df_final_timestampadded_january23.pkl', 'rb') as f:
    df1 = pickle.load(f)
with open('df_final_timestampadded_february23.pkl', 'rb') as f:
    df2 = pickle.load(f)
with open('df_final_timestampadded_march23.pkl', 'rb') as f:
    df3 = pickle.load(f)
with open('df_final_timestampadded_april23.pkl', 'rb') as f:
    df4 = pickle.load(f)
with open('df_final_timestampadded_may23.pkl', 'rb') as f:
    df5 = pickle.load(f)
with open('df_final_timestampadded_june23.pkl', 'rb') as f:
    df6 = pickle.load(f)
with open('df_final_timestampadded_july23.pkl', 'rb') as f:
    df7 = pickle.load(f)
with open('df_final_timestampadded_august23.pkl', 'rb') as f:
    df8 = pickle.load(f)
with open('df_final_timestampadded_october23.pkl', 'rb') as f:
    df10 = pickle.load(f)

In [32]:
#df2.info(verbose=True, show_counts=True)

# 2. COMBINE TRAINING DATA

In [35]:
# TRAINING DATA
dfs = [df1, df2, df3, df4, df5, df6]
# Concatenate all dataframes
df_union_train = pd.concat(dfs, axis=0, ignore_index=True, sort=False)
df_union_train = df_union_train.drop_duplicates()
# Fill missing values with 0
df_union_train.fillna(0, inplace=True)

In [36]:
df_union_train['key_time'] = df_union_train['key'].astype(str) + df_union_train['timestamp'].astype(str)

## ENCODE PATTERN_GRADE (FIT_TRANSFORM)

In [37]:
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

data_encoded = encoder.fit_transform(df_union_train[['pattern_grade']])
data_encoded = pd.DataFrame(data_encoded, columns=encoder.get_feature_names_out(['pattern_grade']))

# Concatenate the encoded columns back to the original DataFrames
df_union_train = pd.concat([df_union_train.reset_index(drop=True), data_encoded], axis=1)

# Drop the original 'pattern_grade' as it's now encoded
df_union_train.drop(columns=['pattern_grade'], axis=1, inplace=True)

c:\Users\nguyenrn\AppData\Local\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [40]:
#df_union_train.info(verbose=True, show_counts=True)

In [41]:
#training data with expiry-related features
df_union_train1 = df_union_train.drop(columns=['key','timestamp','DBMN FP SUPPLY POINT_N.V. NUTRICIA (SP)',
                                               'pattern_grade_DO NOT USE-DO NOT USE-DO NOT USE-DO NOT USE',
                                              'grade_w4_DO NOT USE','grade_w3_DO NOT USE','grade_w2_DO NOT USE',
                                              'grade_w1_DO NOT USE','Country Of Origin_NL']) 

In [72]:
# Separate features and target variable
X1 = df_union_train1.drop('class', axis=1)
y1 = df_union_train1['class']

In [42]:
#training data without expiry-related features
df_union_train2 = df_union_train.drop(columns=['aging','remaining_shelflife','key','timestamp','DBMN FP SUPPLY POINT_N.V. NUTRICIA (SP)',
                                               'pattern_grade_DO NOT USE-DO NOT USE-DO NOT USE-DO NOT USE',
                                              'grade_w4_DO NOT USE','grade_w3_DO NOT USE','grade_w2_DO NOT USE',
                                              'grade_w1_DO NOT USE','Country Of Origin_NL'])

In [73]:
# Separate features and target variable
X2 = df_union_train2.drop('class', axis=1)
y2 = df_union_train2['class']

In [45]:
#df_union_train2.info(verbose=True, show_counts=True)

# 3. TESTING DF7 (HOLDOUT)

In [76]:
#TESTING DATA: USE DF7 first
#dft = [df7, df8, df10]
# Concatenate all dataframes
#df_join = pd.concat(dft, axis=0, ignore_index=True, sort=False)
#df_join = df7.drop_duplicates(inplace=True)

# Fill missing values with 0
df7.fillna(0, inplace=True)
df7['key_time'] = df7['key'].astype(str) + df7['timestamp'].astype(str)

X_holdout_df7 = df7.drop('class', axis=1)
y_holdout_df7 = df7['class']

data_encoded_holdout = encoder.transform(X_holdout_df7[['pattern_grade']])
data_encoded_holdout = pd.DataFrame(data_encoded_holdout, columns=encoder.get_feature_names_out(['pattern_grade']))

# Concatenate the encoded columns back to the original DataFrames
X_holdout_df7 = pd.concat([X_holdout_df7.reset_index(drop=True), data_encoded_holdout], axis=1)

### GROUP 1. EXPIRY

In [77]:
X1_holdout_df7 = X_holdout_df7.drop(columns=['pattern_grade','key','timestamp','DBMN FP SUPPLY POINT_N.V. NUTRICIA (SP)',
                                               'pattern_grade_DO NOT USE-DO NOT USE-DO NOT USE-DO NOT USE',
                                              'grade_w4_DO NOT USE','grade_w3_DO NOT USE','grade_w2_DO NOT USE',
                                              'grade_w1_DO NOT USE','Country Of Origin_NL'])
X1, X1_holdout_df7 = X1.align(X1_holdout_df7, join='outer', axis=1, fill_value=0)

In [88]:
y1_holdout_df7 = y_holdout_df7.copy()

### GROUP 2. NON_EXPIRY

In [80]:

#test data without expiry-related features
X2_holdout_df7 = X_holdout_df7.drop(columns=['pattern_grade','aging','remaining_shelflife','key','timestamp','DBMN FP SUPPLY POINT_N.V. NUTRICIA (SP)',
                                               'pattern_grade_DO NOT USE-DO NOT USE-DO NOT USE-DO NOT USE',
                                              'grade_w4_DO NOT USE','grade_w3_DO NOT USE','grade_w2_DO NOT USE',
                                              'grade_w1_DO NOT USE','Country Of Origin_NL'])

X2, X2_holdout_df7 = X2.align(X2_holdout_df7, join='outer', axis=1, fill_value=0)

In [87]:
y2_holdout_df7 = y_holdout_df7.copy()

In [84]:
'''key_label_counts = df_union_train1.groupby('key_time')['class'].nunique()

# Step 2: Filter to get only keys with exactly 2 unique 'label' values
keys_with_two_labels = key_label_counts[key_label_counts == 2].index

# Step 3: Filter the original DataFrame to only include rows with these keys
filtered_df = df_union_train1[df_union_train1['key'].isin(keys_with_two_labels)]

# Display the resulting DataFrame
filtered_df'''

"key_label_counts = df_union_train1.groupby('key_time')['class'].nunique()\n\n# Step 2: Filter to get only keys with exactly 2 unique 'label' values\nkeys_with_two_labels = key_label_counts[key_label_counts == 2].index\n\n# Step 3: Filter the original DataFrame to only include rows with these keys\nfiltered_df = df_union_train1[df_union_train1['key'].isin(keys_with_two_labels)]\n\n# Display the resulting DataFrame\nfiltered_df"

# 4. TRAIN MODEL RF1 (USED PREVIOUS PARAMETERS)

## 4.1. RF GROUP 1. EXPIRY

In [ ]:
# Step 2: Implement TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

# Placeholder lists for storing performance metrics
accuracies, precisions, recalls, f1_scores = [], [], [], []

# Step 3: Train and validate the model using the splits
for train_index, test_index in tscv.split(X1):
    X_train, X_test = X1.iloc[train_index], X1.iloc[test_index]
    y_train, y_test = y1.iloc[train_index], y1.iloc[test_index]

    # Create and fit the model
    model_rf1 = RandomForestClassifier(bootstrap=True,criterion='entropy',max_depth=None,max_features=None,min_samples_leaf=1,
                                   min_samples_split=15,n_estimators=120,random_state=42)
    model_rf1.fit(X_train, y_train)

    # Predictions
    predictions_rf1 = model_rf1.predict(X_test)

    # Step 4: Evaluate performance
    accuracies.append(accuracy_score(y_test, predictions_rf1))
    precisions.append(precision_score(y_test, predictions_rf1, average='macro'))
    recalls.append(recall_score(y_test, predictions_rf1, average='macro'))
    f1_scores.append(f1_score(y_test, predictions_rf1, average='macro'))

# Display average of performance metrics across all folds
print(f'Average Accuracy: {sum(accuracies) / len(accuracies)}')
print(f'Average Precision: {sum(precisions) / len(precisions)}')
print(f'Average Recall: {sum(recalls) / len(recalls)}')
print(f'Average F1 Score: {sum(f1_scores) / len(f1_scores)}')
print(accuracies)
print(precisions)
print(recalls)
print(f1_scores)

## 4.2. RF2 GROUP 2: NON_EXPIRY

In [ ]:
# Step 2: Implement TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

# Placeholder lists for storing performance metrics
accuracies, precisions, recalls, f1_scores = [], [], [], []

# Step 3: Train and validate the model using the splits
for train_index, test_index in tscv.split(X2):
    X_train, X_test = X2.iloc[train_index], X2.iloc[test_index]
    y_train, y_test = y2.iloc[train_index], y2.iloc[test_index]

    # Create and fit the model
    model_rf2 = RandomForestClassifier(bootstrap=True,criterion='entropy',max_depth=None,max_features=None,min_samples_leaf=1,
                                   min_samples_split=15,n_estimators=120,random_state=42)
    model_rf2.fit(X_train, y_train)

    # Predictions
    predictions_rf2 = model_rf2.predict(X_test)

    # Step 4: Evaluate performance
    accuracies.append(accuracy_score(y_test, predictions_rf2))
    precisions.append(precision_score(y_test, predictions_rf2, average='macro'))
    recalls.append(recall_score(y_test, predictions_rf2, average='macro'))
    f1_scores.append(f1_score(y_test, predictions_rf2, average='macro'))

# Display average of performance metrics across all folds
print(f'Average Accuracy: {sum(accuracies) / len(accuracies)}')
print(f'Average Precision: {sum(precisions) / len(precisions)}')
print(f'Average Recall: {sum(recalls) / len(recalls)}')
print(f'Average F1 Score: {sum(f1_scores) / len(f1_scores)}')
print(accuracies)
print(precisions)
print(recalls)
print(f1_scores)

# 5. PREDICT RF1 ON TEST_RF => RESULT 1

## 5.1. RESULT RF1 (EXPIRY)

In [ ]:
y1_pred_df7 = model_rf1.predict(X1_holdout_df7)

# Generate classification report
report = classification_report(y1_holdout_df7, y1_pred_df7)
print("Classification Report:")
print(report)

## 5.2. RESULT RF2 (NON_EXPIRY)

In [ ]:
y2_pred_df7 = model_rf2.predict(X2_holdout_df7)

# Generate classification report
report = classification_report(y2_holdout_df7, y2_pred_df7)
print("Classification Report:")
print(report)

# 5. FEATURE IMPORTANCE (VERSION BASIC)

# 5.2. FEATURE IMPORTANCE (SHAP)

# 6. OPTIONAL: IF HAVE TIME, RUN OPTUNA AGAIN

## RUN OPTUNA ON GROUP 2 (NON_EXPIRY)

In [ ]:
!pip install optuna
import optuna
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import make_scorer, accuracy_score

# Define the objective function for Optuna
def objective(trial):
    # Suggest hyperparameters to be tuned
    n_estimators = trial.suggest_int('n_estimators', 50, 700)
    max_depth = trial.suggest_int('max_depth', 3, 50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
    bootstrap = trial.suggest_categorical('bootstrap', [True, False])

    
    # Create and configure the RandomForestClassifier model
    model_optuna2 = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        criterion=criterion,
        max_features=max_features,
        bootstrap=bootstrap,
        random_state=42
    )
    
    # Perform TimeSeriesSplit cross-validation
    tscv = TimeSeriesSplit(n_splits=5)
    scoring = {'accuracy': make_scorer(accuracy_score),
               'precision': make_scorer(precision_score, zero_division=0, average='macro'),
               'recall': make_scorer(recall_score, zero_division=0, average='macro'),
               'f1': make_scorer(f1_score, zero_division=0, average='macro')}

    # Perform cross-validation
    results_optuna2 = cross_validate(model_optuna2, X2, y2, cv=tscv, scoring=scoring)

    # Combine results to get an overall average score
    # This example takes a simple average of all scores, adjust weighting as necessary
    avg_recall = np.mean(results_optuna2['test_recall'])

    # Return the negative of the composite score to minimize
    return -avg_recall

# Create an Optuna study and optimize
study_optuna2 = optuna.create_study(direction='minimize')
study_optuna2.optimize(objective, n_trials=50)

# Output the best results
print("Best parameters:", study_optuna2.best_params)
print("Best score: ", -study_optuna2.best_value) 